# TIN Geometry Parsing and Centroid Calculation

In [1]:
# -------------------------------------------------------------------------
# Author: Farid Javadnejad
# Date: 2025-02-05
# Description: 
# This script parses a TIN geometry from a LandXML file, computes triangle areas,
# and calculates the area-weighted centroid of the TIN.
# Functions:
#   - parse_tin_geometry: Parses the XML file to extract points and faces.
#   - compute_triangle_area: Computes the area of a triangle using the cross-product method.
#   - compute_weighted_centroid: Computes the area-weighted centroid of a TIN.
# -------------------------------------------------------------------------

## Libraries

In [2]:
import sys, re
from bs4 import BeautifulSoup as bs
import numpy as np

## Helper Functions

In [10]:
def parse_tin_geometry(xml_file):
    """
    Parses a TIN geometry from a LandXML file and returns points and faces.

    Steps:
    1. Reads the XML file using BeautifulSoup.
    2. Extracts faces from <F> tags, skipping those with i="1"
       read: <F n="5 2 6">9 5 6</F>
       return: [[9, 5, 6]]
    3. Extracts points from <P> tags.
       read: <P id="100">1.0 2.0 3.0</P>
       return: {100: np.array([1.0, 2.0, 3.0])}
    """
    global PRINT_LIMIT
    points = {}
    faces = []

    # Read the XML file & catch any exception
    try:
        with open(xml_file, 'r', encoding='utf-8') as file:
            xml_soup = bs(file, 'lxml-xml')
        print (f"Reading {xml_file}")
    except Exception as e:
        raise ValueError(f"Error reading XML file: {e}")

    # Extract faces
    print("\nExtracting faces...")
    counter = 0
    for face in xml_soup.find_all('F'):
        if face.get("i") == "1":
            continue  # Skip flagged faces
        face_list = []
        for x in face.text.split():
            face_list.append(int(x))
        faces.append(face_list)

        #print data point        
        if counter < PRINT_LIMIT:
            print(face_list)
        counter += 1

    # Extract points
    print("\nExtracting points...")
    counter = 0
    for point_tag in xml_soup.find_all('P'):
        pid = int(point_tag["id"])
        point_list = []
        for x in point_tag.text.split():
            point_list.append(float(x))
        points[pid] = np.array(point_list)

        
        #print data point        
        if counter < PRINT_LIMIT:
            print(point_list)
        counter += 1
    
    # Return points and faces
    return points, faces


def compute_triangle_area(p1, p2, p3):
    """
    Computes the area of a triangle in 3D space using the cross-product method.
    Area = 0.5 * ||v1 x v2|| = 0.5 ||(p2-p1) x (p3-p1)||
    """
    # Compute the vectors from point p1
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate the cross product
    cross_product = np.cross(v1, v2)

    # Compute the area using the magnitude of the cross product
    area = 0.5 * np.linalg.norm(cross_product)
    return area



def compute_weighted_centroid(points, faces):
    """ 
    Computes the area-weighted centroid of a Triangulated Irregular Network (TIN).
    1. Compute the centroid of each triangle
        Ci = (p1 + p2 + p3) / 3
    2. Compute the area of each triangle
        Ai = 0.5 * ||(p2-p1) x (p3-p1)||
    3. Compute the total weighted centroid
        Ctin = Σ(Ai * Ci) / Σ(Ai)
    """
    global PRINT_LIMIT
    total_weighted_centroid = np.zeros(3)
    total_area = 0
    
    print_counter = 0

    print("\nCalculating centroids and areas...")
    for face in faces:
        p1, p2, p3 = points[face[0]], points[face[1]], points[face[2]]
        area = compute_triangle_area(p1, p2, p3)     
        centroid = (p1 + p2 + p3) / 3.0
        total_weighted_centroid += area * centroid
        total_area += area
        
        if print_counter < PRINT_LIMIT:
            print(f"Points: {p1}, {p2}, {p3}")
            print(f"Centroid: {centroid}")
            print(f"Area: {area}")

        print_counter += 1

    print(f"\nTotal Area:{total_area}")
    weighted_centroid = total_weighted_centroid / total_area if total_area else None
    return weighted_centroid

## Main

In [12]:
# Define a global limit for print statements
PRINT_LIMIT = 3

# Set file address (Note: Use "/" OR "\\" instead of "\" for addressing the file path)
xml_file1 = 'C:\\Farid\\gitProjects\\tin_geometry_parsing_and_calculations\\data\\sample\\my_tin.xml'
xmk_file2 = 'C:\\Farid\\gitProjects\\tin_geometry_parsing_and_calculations\\data\\raw\\NM566324_XDTM_SURFACE.xml'

# Load XML and parse data
points, faces = parse_tin_geometry(xmk_file2)

# Compute area-weighted centroid
weighted_centroid = compute_weighted_centroid(points, faces)
print("Area-weighted centroid:", weighted_centroid)


Reading C:\Farid\gitProjects\tin_geometry_parsing_and_calculations\data\raw\NM566324_XDTM_SURFACE.xml

Extracting faces...
[9, 5, 6]
[5, 76, 6]
[77, 76, 5]

Extracting points...
[1822166.6784821365, 1755605.287130769, 6879.726737344359]
[1822158.873187134, 1755614.2865845717, 6879.850892517768]
[1822152.8314114646, 1755620.9763892465, 6879.985318722473]

Calculating centroids and areas...
Points: [1821935.90812934 1755307.11252778    6875.82707548], [1821891.46085999 1755294.81318221    6876.70698853], [1821870.07543035 1755317.94695243    6890.67778186]
Centroid: [1821899.1481399  1755306.62422081    6881.07061529]
Area: 718.8302572071676
Points: [1821891.46085999 1755294.81318221    6876.70698853], [1821935.88908812 1755344.46025342    6877.49552315], [1821870.07543035 1755317.94695243    6890.67778186]
Centroid: [1821899.14179282 1755319.07346269    6881.62676451]
Area: 1143.3157047782163
Points: [1821928.54692072 1755374.6646795     6883.4217891 ], [1821935.88908812 1755344.4602534